# Random Forest

Random forest is a Supervised Machine Learning Algorithm that is used widely in Classification and Regression problems. 

One of the most important features of the Random Forest Algorithm is that it can handle the data set containing continuous variables as in the case of regression and categorical variables as in the case of classification. It usually performs better for classification problems.

### How it works?

The random forest algorithm creates and trains multiple decision trees which are combined to a single response taking their winner vote for classification or the average value  for regression problems.


### What is a Decision Tree? 

From a high level we can define Decision Trees as algorithms that can be used both classification and regression tasks providing a model to make predictions based on a universe of labeled training data.

A decision tree consists of several layers of IF-THEN-ELSE "forks" that are generated automatically during the training phase to fit the provided data.


## Advantages
- A decision tree is easy to understand and interpret.
- Expert opinion and preferences can be included, as well as hard data.
- Can be used with other decision techniques.
- New scenarios can easily be added.

## Disadvantages
- If a decision tree is used for categorical variables with multiple levels, those variables with more levels will have more information gain.

- Calculations can quickly become very complex, although this is usually only a problem if the tree is being created by hand.


## When should I consider using RandomForest

#### Used only for tabular data

Random forests can only work with structured (tabular) data, meaning data that can be represented
as csv or any other similar technique. Unstructured data like images, speech or text cannot be
processed from random forest and most likely they will require some form of a Neural Network like CNN
or RNN for example.

#### Data requirements

Although "by book" Random Forests should be able to handle both missing and nominal
data the library that is widely used in the python world (SkLearn) does not comply
with it and in order to use it we will need to:

- Clean missing values (either removing them or adjusting them)
- Convert nominal data to categorical
- Unlike to NN in random forests there in no need to normalize or scale the data

# Example of Random Forest

Solving the iris dataset using Random Forest results to the following picture:


<img src="./images/iris-decission-tree.png" style="width:100%"/>